In [920]:
from tensorflow import keras
from keras import layers
from keras.datasets import mnist
import numpy as np
import matplotlib.pyplot as plt
from keras import backend as K
import pandas as pd
from sklearn.utils import shuffle
import tensorflow as tf
import xgboost as xgb
import joblib

"""
original_dim = 27
intermediate_dim = 13
latent_dim = 3

inputs = keras.Input(shape=(original_dim,))
x = layers.Dense(intermediate_dim, activation='relu')(inputs)
#x = layers.Dense(intermediate_dim, activation='linear')(inputs)

h = layers.Dense(intermediate_dim, activation='relu')(x)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)



# Create encoder
encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)
#x = layers.Dense(intermediate_dim, activation='linear')(x)

outputs = layers.Dense(original_dim, activation='linear')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')


optimizer=keras.optimizers.Adam(learning_rate=0.001)
vae.compile(optimizer=optimizer)
"""

"\noriginal_dim = 27\nintermediate_dim = 13\nlatent_dim = 3\n\ninputs = keras.Input(shape=(original_dim,))\nx = layers.Dense(intermediate_dim, activation='relu')(inputs)\n#x = layers.Dense(intermediate_dim, activation='linear')(inputs)\n\nh = layers.Dense(intermediate_dim, activation='relu')(x)\nz_mean = layers.Dense(latent_dim)(h)\nz_log_sigma = layers.Dense(latent_dim)(h)\n\n\n\n# Create encoder\nencoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')\n\n# Create decoder\nlatent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')\nx = layers.Dense(intermediate_dim, activation='relu')(latent_inputs)\n#x = layers.Dense(intermediate_dim, activation='linear')(x)\n\noutputs = layers.Dense(original_dim, activation='linear')(x)\ndecoder = keras.Model(latent_inputs, outputs, name='decoder')\n\n# instantiate VAE model\noutputs = decoder(encoder(inputs)[2])\nvae = keras.Model(inputs, outputs, name='vae_mlp')\n\n\noptimizer=keras.optimizers.Adam(learning_rate=0.001)\nva

# AutoEncoder

## Encoder

In [970]:
original_dim = 27
intermediate_dim = 18
intermediate_dim2 = 8
latent_dim = 2

inputs = keras.Input(shape=(original_dim,))
x = layers.Dense(intermediate_dim, activation='relu')(inputs)
x2 = layers.Dense(intermediate_dim2, activation='relu')(x)

h = layers.Dense(intermediate_dim2, activation='relu')(x2)
z_mean = layers.Dense(latent_dim)(h)
z_log_sigma = layers.Dense(latent_dim)(h)


### Sampling Layer

In [971]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z, the vector encoding a digit."""

    def call(self, inputs):
        z_mean, z_log_sigma = inputs
        
        batch = tf.shape(z_mean)[0]
        
        dim = tf.shape(z_mean)[1]
        
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_sigma) * epsilon

z = Sampling()([z_mean, z_log_sigma])

## Decoder & VAE

In [972]:
# Create encoder
encoder = keras.Model(inputs, [z_mean, z_log_sigma, z], name='encoder')

# Create decoder
latent_inputs = keras.Input(shape=(latent_dim,), name='z_sampling')
x = layers.Dense(intermediate_dim2, activation='relu')(latent_inputs)
x = layers.Dense(intermediate_dim, activation='relu')(x)

outputs = layers.Dense(original_dim, activation='linear')(x)
decoder = keras.Model(latent_inputs, outputs, name='decoder')

# instantiate VAE model
outputs = decoder(encoder(inputs)[2])
vae = keras.Model(inputs, outputs, name='vae_mlp')

## Loss

In [973]:
#mse = keras.losses.MeanSquaredError()
#reconstruction_loss = mse(inputs, outputs)
#reconstruction_loss *= 27
reconstruction_loss_factor = 1000
reconstruction_loss = tf.keras.backend.mean(tf.keras.backend.square(inputs-outputs))

kl_loss = 1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma)
kl_loss = K.sum(kl_loss, axis=-1)
kl_loss *= -0.5

vae_loss = K.mean((reconstruction_loss_factor*reconstruction_loss) + kl_loss)
vae.add_loss(vae_loss)

## Data

In [974]:
dataset = pd.read_csv('datasets/Fs_B_O_DANCE_WALK_KIN_0.5sec.csv')

train_dataset = dataset.sample(frac=0.85, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

print("No Training Samples:",train_dataset.shape[0])
print("No Test Samples:",test_dataset.shape[0])

train_dataset = shuffle(train_dataset)
test_dataset = shuffle(test_dataset)

train_emotions = pd.concat([train_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
test_emotions = pd.concat([test_dataset.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)


train_dataset = np.asarray(train_dataset)
test_dataset = np.asarray(test_dataset)

x_train = train_dataset.reshape((len(train_dataset), np.prod(train_dataset.shape[1:])))
x_test = test_dataset.reshape((len(test_dataset), np.prod(test_dataset.shape[1:])))

print(len(x_train[0]))

No Training Samples: 34156
No Test Samples: 6028
27


## Train

In [975]:
optimizer=keras.optimizers.Adam(learning_rate=0.001)
vae.compile(optimizer=optimizer)

In [976]:
vae.fit(x_train, x_train,
        epochs=1024,
        batch_size=32,
       )

Epoch 1/256
1068/1068 [==============================] - 2s 1ms/step - loss: 118.2572
Epoch 2/256
1068/1068 [==============================] - 2s 1ms/step - loss: 64.9460
Epoch 3/256
1068/1068 [==============================] - 2s 1ms/step - loss: 62.8005
Epoch 4/256
1068/1068 [==============================] - 2s 1ms/step - loss: 60.8321
Epoch 5/256
1068/1068 [==============================] - 2s 1ms/step - loss: 59.7017
Epoch 6/256
1068/1068 [==============================] - 2s 1ms/step - loss: 58.7942
Epoch 7/256
1068/1068 [==============================] - 2s 1ms/step - loss: 58.0960
Epoch 8/256
1068/1068 [==============================] - 2s 1ms/step - loss: 57.5725
Epoch 9/256
1068/1068 [==============================] - 2s 1ms/step - loss: 57.1308
Epoch 10/256
1068/1068 [==============================] - 2s 1ms/step - loss: 56.8623
Epoch 11/256
1068/1068 [==============================] - 2s 1ms/step - loss: 56.5852
Epoch 12/256
1068/1068 [==============================] - 2s 1

KeyboardInterrupt: 

## Test

### Sample Comparison & Emotion Classification

In [938]:
sample = np.asarray(x_test[1])
sample = sample.reshape(1,-1)
print(sample)

[[ 0.8208893   0.57730774  0.49820339  0.53456149  0.42063105  0.40112175
   0.35034314  0.30452922  0.286151    0.27877407  0.06687857 -0.2828076
  -0.23700152  0.92652771  0.57908081  0.09135365  0.22280849  0.31328745
   0.49815666  0.50930095  0.4666077   0.44527651  1.93531632  2.26792412
   1.16245564  1.10699938  1.00147062]]


In [939]:
mean, var, generated = encoder.predict(sample)
print(generated)

[[1.1441323  0.83310235 1.0150403 ]]


In [940]:
regen = decoder.predict(generated)
print(regen)

[[ 0.70034444  0.4066999   0.434201    0.42914027  0.43174216  0.4411949
   0.34746057  0.3517979   0.2860169   0.2765127  -0.0110607  -0.04170739
  -0.10273481  0.93392384  0.3909298   0.12780224  0.16335769  0.70696145
   0.82295746  0.42953163  0.4924381   0.4652107   1.7890185   2.2226584
   0.8636538   1.1588837   0.90664303]]


In [941]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("../../emotion_classifier/model_training/models/l2p_dance_model_O.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("../../emotion_classifier/model_training/models/l2a_dance_model_O.json")

model_d = xgb.XGBRegressor(verbosity=0)
model_d.load_model("../../emotion_classifier/model_training/models/l2d_dance_model_O.json")

scaler = joblib.load('../../emotion_classifier/model_training/datasets/scalers/standardizers/Fs_B_O_S_DANCE_WALK_KIN_0.5sec.pkl') 

In [942]:
scaled_sample = scaler.transform(sample)

real_coordinates = (
    model_p.predict(scaled_sample),
    model_a.predict(scaled_sample),
    model_d.predict(scaled_sample)
)

scaled_regen = scaler.transform(regen)

generated_coordinates = (
    model_p.predict(scaled_regen),
    model_a.predict(scaled_regen),
    model_d.predict(scaled_regen)
)


print('Real: %s' % np.asarray(real_coordinates))
print('Predicted: %s' % np.asarray(generated_coordinates))

Real: [[-0.49220023]
 [ 0.5992359 ]
 [ 0.88934654]]
Predicted: [[0.02798416]
 [0.16476487]
 [0.02670545]]


/home/diogosilva/.local/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
/home/diogosilva/.local/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


### MAE & MSE

In [943]:
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error

In [944]:
mean, var, generated = encoder.predict(x_test)
regen = decoder.predict(generated)

print(regen)

[[0.7036592  0.4342553  0.37454098 ... 1.8461361  2.5204012  1.7883573 ]
 [0.7003445  0.40669996 0.434201   ... 0.8636538  1.1588837  0.9066426 ]
 [0.6201762  0.37764835 0.40436462 ... 0.6113868  0.66184396 0.60663366]
 ...
 [0.70893335 0.4264102  0.42156953 ... 0.7753787  0.6809969  0.8165428 ]
 [0.6548151  0.41043064 0.36380374 ... 1.3698651  2.6179557  1.2896614 ]
 [0.5788953  0.3778478  0.38853133 ... 1.4338753  1.2692828  2.2325377 ]]


In [945]:
mae_errors = mean_absolute_error(x_test, regen, multioutput='raw_values')
mse_errors = mean_squared_error(x_test, regen, multioutput='raw_values')

features = ["max_hand_distance",
          "avg_l_hand_hip_distance",
          "avg_r_hand_hip_distance",
          "max_stride_length",
          "avg_l_hand_chest_distance",
          "avg_r_hand_chest_distance",
          "avg_l_elbow_hip_distance",
          "avg_r_elbow_hip_distance",
          "avg_chest_pelvis_distance",
          "avg_neck_chest_distance",
          "avg_neck_rotation_w", "avg_neck_rotation_x", "avg_neck_rotation_y", "avg_neck_rotation_z",
          "avg_total_body_volume",
          "avg_triangle_area_hands_neck",
          "avg_triangle_area_feet_hips",
          
          "l_hand_speed",
          "r_hand_speed",
          "l_foot_speed",
          "r_foot_speed",
          "neck_speed",
          
          "l_hand_acceleration_magnitude",
          "r_hand_acceleration_magnitude",
          "l_foot_acceleration_magnitude",
          "r_foot_acceleration_magnitude",
          "neck_acceleration_magnitude",
         ]

print("Overall MAE: " + str(mean_absolute_error(x_test, regen)))

print()
for i in range(len(mse_errors)):
    print("==" + features[i] + "==")
    print("MSE: %.5f" % mse_errors[i])
    print("MAE: %.5f" % mae_errors[i])
    print("Example [Regen-Real]: " + str(regen[i][i]) + " - " + str(x_test[i][i]))
    print()

Overall MAE: 0.1102329282097022

==max_hand_distance==
MSE: 0.04548
MAE: 0.16615
Example [Regen-Real]: 0.7036592 - 0.8239585819804882

==avg_l_hand_hip_distance==
MSE: 0.01716
MAE: 0.10435
Example [Regen-Real]: 0.40669996 - 0.5773077437283716

==avg_r_hand_hip_distance==
MSE: 0.02028
MAE: 0.11390
Example [Regen-Real]: 0.40436462 - 0.5239518888099561

==max_stride_length==
MSE: 0.01787
MAE: 0.10228
Example [Regen-Real]: 0.3245529 - 0.1501770187519039

==avg_l_hand_chest_distance==
MSE: 0.01053
MAE: 0.08080
Example [Regen-Real]: 0.40303537 - 0.459039720444281

==avg_r_hand_chest_distance==
MSE: 0.01036
MAE: 0.07894
Example [Regen-Real]: 0.41420522 - 0.394466162432962

==avg_l_elbow_hip_distance==
MSE: 0.00471
MAE: 0.05190
Example [Regen-Real]: 0.31152064 - 0.3305304711001226

==avg_r_elbow_hip_distance==
MSE: 0.00482
MAE: 0.05167
Example [Regen-Real]: 0.34142977 - 0.4395373059793347

==avg_chest_pelvis_distance==
MSE: 0.00000
MAE: 0.00010
Example [Regen-Real]: 0.28611562 - 0.286150999913

# PAD to Latent Space

In [946]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.lines as mlines

import tensorflow as tf

import sklearn
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold

import xgboost as xgb

xgb.set_config(verbosity=0)

from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import normalize

from sklearn.utils import shuffle
import math
import joblib

from datetime import datetime
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
import time


## Create Dataset

In [947]:
model_p = xgb.XGBRegressor(verbosity=0)
model_p.load_model("../../emotion_classifier/model_training/models/l2p_dance_model_O.json")

model_a = xgb.XGBRegressor(verbosity=0)
model_a.load_model("../../emotion_classifier/model_training/models/l2a_dance_model_O.json")

model_d = xgb.XGBRegressor(verbosity=0)
model_d.load_model("../../emotion_classifier/model_training/models/l2d_dance_model_O.json")

scaler = joblib.load('../../emotion_classifier/model_training/datasets/scalers/standardizers/Fs_B_O_S_DANCE_WALK_KIN_0.5sec.pkl') 

## Data

In [948]:
# X -> Predicted emotions of LMA features
# y -> Latent Space of LMA features
# Good Error: Predicted emotion of LMA features generated from latent space == Predicted emotions of original LMA features

dataset = pd.read_csv('../../emotion_classifier/model_training/datasets/Fs_B_O_S_DANCE_WALK_KIN_0.5sec.csv')

train_dataset = dataset.sample(frac=0.85, random_state=42)
test_dataset = dataset.drop(train_dataset.index)

print("No Training Samples:",train_dataset.shape[0])
print("No Test Samples:",test_dataset.shape[0])

train_lma = train_dataset.copy()
test_lma = test_dataset.copy()

train_emotions = pd.concat([train_lma.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)
test_emotions = pd.concat([test_lma.pop(x) for x in ['EMOTION_P', 'EMOTION_A', 'EMOTION_D']], axis=1)

No Training Samples: 34156
No Test Samples: 6028


In [949]:
train_X_p = model_p.predict(train_lma)
train_X_a = model_a.predict(train_lma)
train_X_d = model_d.predict(train_lma)

rows = []
for i in range(len(train_lma)):
    rows.append([train_X_p[i], train_X_a[i], train_X_d[i]])

train_X = pd.DataFrame(rows, columns=[
            "EMOTION_P", "EMOTION_A", "EMOTION_D"
         ])

train_X.head()

,EMOTION_P,EMOTION_A,EMOTION_D
0,0.596379,0.498261,0.199826
1,0.699091,0.199438,0.199085
2,-0.586425,-0.288315,-0.289255
3,-0.350705,0.698464,-0.799689
4,-0.351387,0.699263,-0.800516


In [950]:
test_X_p = model_p.predict(test_lma)
test_X_a = model_a.predict(test_lma)
test_X_d = model_d.predict(test_lma)

rows = []
for i in range(len(test_lma)):
    rows.append([test_X_p[i], test_X_a[i], test_X_d[i]])

test_X = pd.DataFrame(rows, columns=[
            "EMOTION_P", "EMOTION_A", "EMOTION_D"
         ])

test_X.head()

,EMOTION_P,EMOTION_A,EMOTION_D
0,-0.499540,0.599887,0.896598
1,0.599478,0.496702,0.201141
2,-0.581306,-0.292542,-0.287728
3,-0.602087,-0.301238,-0.288370
4,-0.595353,-0.301161,-0.302442


In [951]:
mean, var, latent_space = encoder.predict(train_lma)

rows = []
for i in range(len(latent_space)):
    rows.append([latent_space[i][0], latent_space[i][1], latent_space[i][2]])
    
train_y = pd.DataFrame(rows, columns=[
            "LATENT_1", "LATENT_2" #, "LATENT_3"
         ])

train_y_1 = pd.concat([train_y.pop(x) for x in ['LATENT_1']], axis=1)
train_y_2 = pd.concat([train_y.pop(x) for x in ['LATENT_2']], axis=1)
#train_y_3 = pd.concat([train_y.pop(x) for x in ['LATENT_3']], axis=1)

train_y_1.head()

,LATENT_1
0,0.751822
1,0.582938
2,1.040619
3,0.569951
4,0.689253


In [952]:
mean, var, latent_space = encoder.predict(test_lma)

rows = []
for i in range(len(latent_space)):
    rows.append([latent_space[i][0], latent_space[i][1], latent_space[i][2]])
    
test_y = pd.DataFrame(rows, columns=[
            "LATENT_1", "LATENT_2" #, "LATENT_3"
         ])

test_y_1 = pd.concat([test_y.pop(x) for x in ['LATENT_1']], axis=1)
test_y_2 = pd.concat([test_y.pop(x) for x in ['LATENT_2']], axis=1)
#test_y_3 = pd.concat([test_y.pop(x) for x in ['LATENT_3']], axis=1)

test_y_1.head()

,LATENT_1
0,1.194246
1,0.949085
2,1.248255
3,1.008567
4,0.994421


## Train Regressor

In [953]:
model_1 = xgb.XGBRegressor(
                    n_estimators=1500, learning_rate=0.05, max_depth=10, min_child_weight=5, 
                    reg_alpha=0.1, reg_lambda=1, gamma=0.0,
                    subsample=0.75, colsample_bytree=0.75, objective="reg:squarederror",
                    tree_method='gpu_hist'
                )

model_2 = xgb.XGBRegressor(
                    n_estimators=1500, learning_rate=0.05, max_depth=10, min_child_weight=5, 
                    reg_alpha=0.1, reg_lambda=1, gamma=0.0,
                    subsample=0.75, colsample_bytree=0.75, objective="reg:squarederror",
                    tree_method='gpu_hist'
                )

#model_3 = xgb.XGBRegressor(
#                    n_estimators=1500, learning_rate=0.05, max_depth=10, min_child_weight=5, 
#                    reg_alpha=0.1, reg_lambda=1, gamma=0.0,
#                    subsample=0.75, colsample_bytree=0.75, objective="reg:squarederror",
#                    tree_method='gpu_hist'
#                )

In [954]:
model_1.fit(train_X, train_y_1)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.75,
             enable_categorical=False, gamma=0.0, gpu_id=0,
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=10,
             min_child_weight=5, missing=nan, monotone_constraints='()',
             n_estimators=1500, n_jobs=12, num_parallel_tree=1,
             predictor='auto', random_state=0, reg_alpha=0.1, reg_lambda=1,
             scale_pos_weight=1, subsample=0.75, tree_method='gpu_hist',
             validate_parameters=1, verbosity=None)

In [955]:
model_2.fit(train_X, train_y_2)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.75,
             enable_categorical=False, gamma=0.0, gpu_id=0,
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=10,
             min_child_weight=5, missing=nan, monotone_constraints='()',
             n_estimators=1500, n_jobs=12, num_parallel_tree=1,
             predictor='auto', random_state=0, reg_alpha=0.1, reg_lambda=1,
             scale_pos_weight=1, subsample=0.75, tree_method='gpu_hist',
             validate_parameters=1, verbosity=None)

In [956]:
#model_3.fit(train_X, train_y_3)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=0.75,
             enable_categorical=False, gamma=0.0, gpu_id=0,
             importance_type=None, interaction_constraints='',
             learning_rate=0.05, max_delta_step=0, max_depth=10,
             min_child_weight=5, missing=nan, monotone_constraints='()',
             n_estimators=1500, n_jobs=12, num_parallel_tree=1,
             predictor='auto', random_state=0, reg_alpha=0.1, reg_lambda=1,
             scale_pos_weight=1, subsample=0.75, tree_method='gpu_hist',
             validate_parameters=1, verbosity=None)

## Test Model

In [957]:
score = model_1.score(train_X, train_y_1)  

print("Latent 1")
print("Training score: ", score)

print()

score = model_2.score(train_X, train_y_2)  

print("Latent 2")
print("Training score: ", score)

score = model_3.score(train_X, train_y_3)  

print()

#print("Latent 3")
#print("Training score: ", score)

Latent 1
Training score:  0.5749038956031831

Latent 2
Training score:  0.5931620956821015

Latent 3
Training score:  0.5885978770571259


In [958]:
pred_y_1 = model_1.predict(test_X)
mse = mean_squared_error(test_y_1, pred_y_1)
mae = mean_absolute_error(test_y_1, pred_y_1)
print("Latent 1")
print("MSE: %.2f" % mse)
print("MAE: %.2f" % mae)


pred_y_2 = model_2.predict(test_X)
mse = mean_squared_error(test_y_2, pred_y_2)
mae = mean_absolute_error(test_y_2, pred_y_2)
print("\nLatent 2")
print("MSE: %.2f" % mse)
print("MAE: %.2f" % mae)


#pred_y_3 = model_3.predict(test_X)
#mse = mean_squared_error(test_y_3, pred_y_3)
#mae = mean_absolute_error(test_y_3, pred_y_3)
#print("\nLatent 3")
#print("MSE: %.2f" % mse)
#print("MAE: %.2f" % mae)

Latent 1
MSE: 0.19
MAE: 0.31

Latent 2
MSE: 0.08
MAE: 0.20

Latent 3
MSE: 0.15
MAE: 0.27


In [969]:
index = 10

# Real -> PAD coordinates
#   - get latent space
#   - convert from latent to LMA features
#   - standardize
#   - predict coordinates of generated features

sample = np.asarray([test_X.iloc[index]])
latent = np.asarray([[model_1.predict(sample)[0], model_2.predict(sample)[0]]])#, model_3.predict(sample)[0]]])


generated_lma = decoder.predict(latent)


scaled_gen = scaler.transform(generated_lma)

generated_coord = (
    model_p.predict(scaled_gen)[0],
    model_a.predict(scaled_gen)[0],
    model_d.predict(scaled_gen)[0]
)

print('Real: %s' % sample[0])
print('Predicted: %s' % np.asarray(generated_coord))

Real: [-0.5046277   0.5842134   0.86978376]
Predicted: [-0.27994117 -0.0327893  -0.37819585]


/home/diogosilva/.local/lib/python3.7/site-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"


## RandomSearchCV

In [939]:
# A parameter grid for XGBoost
# https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/
params = {
        'eta': [0.01, 0.05, 0.1],
        'min_child_weight': [1, 5, 11, 21],
        'max_depth': [3, 6, 10, 15],
        'gamma': [0, 0.001, 0.01],
        'subsample': [0.75, 1],
        'colsample_bytree': [0.75, 1],
        'lambda': [1, 1.25],
        'alpha': [0.0, 0.25]
        }

n_iter = 150

In [940]:
model_1 = xgb.XGBRegressor(
                    n_estimators=1500,
                    objective="reg:squarederror",
                    tree_method='gpu_hist'
                )

model_2 = xgb.XGBRegressor(
                    n_estimators=1500,
                    objective="reg:squarederror",
                    tree_method='gpu_hist'
                )

In [941]:
# Latent 1
# run randomized search
kfold = KFold(n_splits=5, shuffle=True)

random_search_1 = RandomizedSearchCV(model_1, param_distributions=params,
                               cv=kfold, scoring='neg_mean_squared_error', n_iter = n_iter)

start = time.time()
random_search.fit(train_X, train_y_1)


print("GridSearchCV took %.2f seconds"
      " parameter settings." % ((time.time() - start)))

GridSearchCV took 2863.87 seconds parameter settings.


In [ ]:
# Latent 2
# run randomized search
kfold = KFold(n_splits=5, shuffle=True)

random_search_2 = RandomizedSearchCV(model_2, param_distributions=params,
                               cv=kfold, scoring='neg_mean_squared_error', n_iter = n_iter)

start = time.time()
random_search.fit(train_X, train_y_2)


print("GridSearchCV took %.2f seconds"
      " parameter settings." % ((time.time() - start)))

### Results

In [942]:
best_regressor_1 = random_search_1.best_estimator_

print(best_regressor_1.get_params())

{'objective': 'reg:squarederror', 'base_score': 0.5, 'booster': 'gbtree', 'colsample_bylevel': 1, 'colsample_bynode': 1, 'colsample_bytree': 1, 'enable_categorical': False, 'gamma': 0.001, 'gpu_id': 0, 'importance_type': None, 'interaction_constraints': '', 'learning_rate': 0.00999999978, 'max_delta_step': 0, 'max_depth': 3, 'min_child_weight': 5, 'missing': nan, 'monotone_constraints': '()', 'n_estimators': 1500, 'n_jobs': 12, 'num_parallel_tree': 1, 'predictor': 'auto', 'random_state': 0, 'reg_alpha': 0.25, 'reg_lambda': 1, 'scale_pos_weight': 1, 'subsample': 0.75, 'tree_method': 'gpu_hist', 'validate_parameters': 1, 'verbosity': None, 'lambda': 1, 'eta': 0.01, 'alpha': 0.25}


In [ ]:
best_regressor_1 = random_search_1.best_estimator_

print(best_regressor_1.get_params())

In [943]:
pred_y = best_regressor.predict(reg_test_X)
mse = mean_squared_error(reg_test_Y, pred_y)
mae = mean_absolute_error(reg_test_Y, pred_y)

print("MSE: %.2f" % mse)
print("MAE: %.2f" % mae)

MSE: 0.98
MAE: 0.78
